In [ ]:
import boto3, re
from sagemaker import get_execution_role

Define the IAM role of execution

In [ ]:
role = get_execution_role()

Loads keras libraries

In [ ]:
import keras
from keras.models import model_from_json


Creates directory with keras models

In [ ]:
!mkdir keras_model

In [ ]:
!ls keras_model
!pwd

In [ ]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'sound_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/sound_model.h5')
print("Loaded model of feature extraction from disk")
loaded_model.summary()

In [ ]:
!rm -rf export

# Export the Keras model (sound extractor) to the TensorFlow ProtoBuf format

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [ ]:
from keras import backend as K

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/'+ model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"vgg_features": loaded_model.output})

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

In [ ]:
!ls export/Servo/1

In [ ]:
!ls export/Servo/1/variables

In [ ]:
import tarfile
with tarfile.open('sound_extractor.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='sound_extractor.tar.gz', key_prefix='sound_extractor')

Creates an empty file

In [ ]:
!touch train.py

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/sound_extractor/sound_extractor.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [ ]:
import boto3
s3 = boto3.resource('s3')

my_bucket = s3.Bucket(sagemaker_session.default_bucket())

for file in my_bucket.objects.all():
    print(file.key)

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.p3.2xlarge')

In [ ]:
!rm -rf export

# Does same thing for classifier model

In [ ]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'sound_class.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/sound_class.h5')
print("Loaded model of feature extraction from disk")
loaded_model.summary()

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
from keras import backend as K

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/'+ model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"output": loaded_model.output})

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

In [ ]:
!ls export/Servo/1

In [ ]:
!ls export/Servo/1/variables

In [ ]:
import tarfile
with tarfile.open('sound_classifier.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='sound_classifier.tar.gz', key_prefix='sound_classifier')

In [ ]:
import boto3
s3 = boto3.resource('s3')

my_bucket = s3.Bucket(sagemaker_session.default_bucket())

for file in my_bucket.objects.all():
    print(file.key)

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/sound_classifier/sound_classifier.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

## Download samples of acoustic scenes for testing

In [ ]:
!wget https://zenodo.org/record/1040168/files/TUT-acoustic-scenes-2017-evaluation.audio.1.zip

In [ ]:
!unzip TUT-acoustic-scenes-2017-evaluation.audio.1.zip

In [ ]:
!pip install --upgrade pip
!pip install resampy pysoundfile SoundFile

In [ ]:
!pwd

In [ ]:
import numpy as np
from scipy.io import wavfile
import six
import tensorflow as tf
import csv
import datetime
import numpy as np
import argparse
import csv
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/ml_osoai/audioset_sagemaker/audioset_feature_extractor')
import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

In [ ]:
wav_file = './TUT-acoustic-scenes-2017-evaluation/audio/10.wav'
examples_batch = wavfile_to_examples(wav_file)